# Simple Proof of Work Example
Sample bitcoin `block number` (or `height`) 650750. This is a recent block from September 30th.

In [1]:
blockNum = 650750

Lots of sources and APIs. For this example I used *Blockcypher*

In [ ]:
import os
os.system("curl 'https://api.blockcypher.com/v1/btc/main/blocks/"+blockNum  +" > block" + blockNum + ".json")

Load the file and take a look at fields (API providers include extra info, variations)

In [3]:
import json

with open('block650750.json') as f:
    d = json.load(f)

d.keys()

dict_keys(['hash', 'height', 'chain', 'total', 'fees', 'size', 'ver', 'time', 'received_time', 'coinbase_addr', 'relayed_by', 'bits', 'nonce', 'n_tx', 'prev_block', 'mrkl_root', 'txids', 'depth', 'prev_block_url', 'tx_url', 'next_txids'])

## Proof of Work Computation
This uses 6 fields from the block header
1. version (4 bytes)
2. hash of previous block (32 bytes)
3. Merkle root (of transactions included in the block) (32 bytes)
4. block time stamp (4 bytes)
5. bits (4 bytes)
6. nonce (4 bytes)

Proof of work computation
* concatatenate the above
* get the **double** sha256 hash

The result should be the same as the block's hash

In [4]:
# 6 components
version = d['ver'] #536870912
hashPrevBlock = d['prev_block']  #"00000000000000000001bcd7e222ebf6b1332f49b1524721dfd3673599960464"
hashMerkleRoot = d['mrkl_root'] #"d34949770bcdbd1a20db0192e0795135696c2eb6668ba5d831df2dc87c37b5ba" 
blkTimeStamp =	d['time'] #"2020-10-01T03:11:34Z"
bits = d['bits'] # 386831018
nonce = d['nonce'] #25376585

#block hash
blk_hash = d['hash'] #"0000000000000000000ca0a08f726db261675912567d7dbfff0d07d25bf05925"

We've got 3 integers (version, bits, nonce) and 3 strings (timestamp, prevHash, Merkle Root)

Before computing the block hash (PoW), we need to convert them all 
* Hex Strings (simple)
* Little **endian** (i had to look this up for Python)

## Some helper functions

### Convert 4 byte integer to Little endian Hex strings
This is for **version, bits** and **nonce**

In [5]:
def field2LilEndHex(s):
    return s.to_bytes(4,'little').hex()

For example, try version number

In [8]:
version

536870912

In [10]:
hex_ver = field2LilEndHex(version)
hex_ver

'00000020'

### Convert hashes from Big to Little Endian
This is for the **hash of previous block** and **Merkle root**

In [11]:
def Hash2LilEnd(h):
    return int(h,16).to_bytes(32, 'little').hex()

### Convert timestamp string to unix time and Little Endian  

In [12]:
import time
from datetime import datetime

def time2LilEndHex(t):
    t = t.replace("Z","")
    t1 = datetime.fromisoformat(t)
    utc = int(time.mktime(t1.timetuple())) - 7*3600 #unix time - time delta with UTC
    return utc.to_bytes(4,'little').hex()

For example

In [13]:
time2LilEndHex("2011-05-21T17:26:31")

'c7f5d74d'

Convert all the fields into Hex strings using the above helper functions

In [15]:
## convert the fields into Hex with appropriate endian-ness
hex_ver = field2LilEndHex(version)
hex_prevHash = Hash2LilEnd(hashPrevBlock)
hex_MerkleRoot = Hash2LilEnd(hashMerkleRoot)
hex_timestamp = time2LilEndHex(blkTimeStamp)
hex_bits = field2LilEndHex(bits)
hex_nonce = field2LilEndHex(nonce)

Add them all and **convert to Big endian before** hashing

In [16]:
header_hex_concat = hex_ver + hex_prevHash + hex_MerkleRoot + hex_timestamp + hex_bits + hex_nonce 
## convert to bytes with Big endian
header_bin = int(header_hex_concat,16).to_bytes(80,'big')

## Compute double SHA256

In [17]:
import hashlib

hash = hashlib.sha256(hashlib.sha256(header_bin).digest()).digest()

Convert binary hash to Hex String

In [19]:
hex_hash = hash.hex()

In [20]:
hex_hash

'2559f05bd2070dffbf7d7d5612596761b26d728fa0a00c000000000000000000'

This is big endian! We need the 0's out in front for Proof of Work

In [22]:
hex_hash_Rev = Hash2LilEnd(hex_hash)
hex_hash_Rev

'0000000000000000000ca0a08f726db261675912567d7dbfff0d07d25bf05925'

### Verify it matches the block header

In [23]:
hex_hash_Rev == blk_hash

True

**Summary:** 
* This is how we go from `blockHeader -> blockHash`
* If the hash meets the **target difficulty (enough 0's)**, then it serves as Proof of work

The bitcoin network is performing about 150 Exa Hash (150 * 10^18) 
such computations per second.

To overcome this network (51% attack), you need to amass and sustain the computing power of about 75 billion-billion Block Hash computations.
